# Project 3

<b> Group members: </b>
1.	Dianela Perdomo 
2.	Hendyel Pacheco
3.	Lihe Liu
4.	Pornpamol Pattamanont

## Data:
The data file for this problem set is here:
```bash
/ufrc/zoo6927/share/Class_Files/data/flights.May2017-Apr2018.csv 
```

## Plan:

1. Two tables we are going to create are:
    * Delayed flights depart from GNV 
    * Distance of those flights with airlines 
2. The first table has 6 columns: 
    * FL_DATE /AIRLINE_ID (this will be the foreign key that we will add) / ORIGIN / ORIGIN_CITY_NAME / DEST / DEP_DEL15. 
    * The primary key in the first table is a number.
3. The second table is ‘Airlines table’ which has:
    * AIRLINE_ID / DISTANCE/ORIGIN_CITY_NAME / DEST.
    * The key in this table is our foreign key in first table.
4. We use 'join' function to link between these 2 tables; where to check GNV in table 2.
5. We will analyze how many delayed flights each airline has, and organize from max to min.




## Solution:

<b> Create empty database: </b>

In [3]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import DateTime, Boolean
from sqlalchemy import exists
from sqlalchemy import sql, select, join, desc

# Create a sqlite database 
engine = create_engine('sqlite:///pj3_flights.sqlite')

metadata = MetaData(engine)

# Try to load Airports info from database, if not there, create it.
try:
    Airports=Table('Airports', metadata, autoload=True)
except:
    Airports = Table ('Airports', metadata,
                Column('ID', Integer, autoincrement=True),
                Column('Code', String, primary_key=True),
                Column('City', String),
                Column('State', String),
                Column('Name', String),
               )

# Same for Flights table.
try:
    Flights=Table('Flights', metadata, autoload=True)
except:
    Flights = Table ('Flights', metadata,
                 Column('Fl_date', DateTime),
                 Column('Airline_ID', String),
                 Column('Origin', String, ForeignKey("Airports.Code")),
                 Column('Destination', String, ForeignKey("Airports.Code")),
                 Column('Dep_Time', String),
                 Column('Dep_Delay_New', Integer),
                 Column('Arr_Time', String),
                 Column('Arr_Delay_New', Integer),
                 Column('Cancelled', Boolean),
                 Column('Cancellation_Code', String),
                 Column('Diverted', Boolean),
                 Column('Air_Time', String),
                 Column('Flights', Integer),
                 Column('Distance', Integer),
                 Column('Carrier_Delay', Integer),
                 Column('Weather_Delay', Integer),
                 Column('NAS_Delay', Integer),
                 Column('Security_Delay', Integer),
                 Column('Late_Aircraft_Delay', Integer)
                )
 

metadata.create_all(engine)


<b> Put the Flights data into dataframe: </b>

In [29]:
import datetime
import pandas as pd
import csv

def to_date(dates, lookup=False, **args):
    if lookup:
        return dates.map({v: pd.to_datetime(v, **args) for v in dates.unique()})
    return pd.to_datetime(dates, **args)

conn = engine.connect()

# Open to get flight data

flights=open("/ufrc/zoo6927/share/Class_Files/data/flights.1K.csv")

reader = csv.DictReader(flights)

for Line in reader:

    ins=Flights.insert().values(Fl_date=to_date(Line['FL_DATE']),
                                Airline_ID = Line['AIRLINE_ID'],
                                Origin = Line['ORIGIN'],
                                Destination = Line['DEST'],
                                Dep_Time = Line['DEP_TIME'],
                                Dep_Delay_New = Line['DEP_DELAY_NEW'],
                                Arr_Time = Line['ARR_TIME'],
                                Arr_Delay_New = Line['ARR_DELAY_NEW'],
                                Cancelled = int(float((Line['CANCELLED']))),
                                Cancellation_Code = Line['CANCELLATION_CODE'],
                                Diverted = int(float((Line['DIVERTED']))),
                                Air_Time = Line['AIR_TIME'],
                                Flights = Line['FLIGHTS'],
                                Distance = Line['DISTANCE'],
                                Carrier_Delay = Line['CARRIER_DELAY'],
                                Weather_Delay = Line['WEATHER_DELAY'],
                                NAS_Delay = Line['NAS_DELAY'],
                                Security_Delay = Line['SECURITY_DELAY'],
                                Late_Aircraft_Delay = Line['LATE_AIRCRAFT_DELAY']
                                          )
    result = conn.execute(ins)

## Select, Where and Join

**Select all origin and destination pairs, limit = 10**


In [36]:
from sqlalchemy import select

query1 = select([Flights.c.Origin, Flights.c.Destination])\
        .limit(10)
             
result = conn.execute(query1)
for row in result:
    print(row)

('LAX', 'IAD')
('LAX', 'IAD')
('SAN', 'SFO')
('SFO', 'EWR')
('JFK', 'SFO')
('LAX', 'OGG')
('OGG', 'LAX')
('SFO', 'JFK')
('LAX', 'EWR')
('EWR', 'LAX')


**Count the total number of flights delayed more than 15 minutes**

In [46]:
from sqlalchemy import select
from sqlalchemy.sql.expression import func

#how to do count

query2 = select([func.count(Flights.c.Dep_Delay_New)])\
        .where(Flights.c.Dep_Delay_New > 15)
             
result = conn.execute(query2)
for row in result:
    print(row)

(247,)


**Count the total number of flights delayed more than 15 minutes by Origin**

In [48]:
from sqlalchemy import select
from sqlalchemy.sql.expression import func

#Count 

query3 = select([Flights.c.Origin, func.count(Flights.c.Dep_Delay_New)])\
        .where(Flights.c.Dep_Delay_New > 15)\
        .group_by(Flights.c.Origin)
             
result = conn.execute(query3)
for row in result:
    print(row)
    


('ALB', 1)
('ATL', 4)
('AUS', 1)
('AVL', 1)
('BOS', 9)
('BUR', 1)
('CMH', 1)
('DAL', 3)
('DCA', 1)
('DEN', 21)
('DFW', 2)
('EWR', 22)
('FLL', 2)
('HNL', 1)
('IAD', 13)
('IAH', 9)
('JFK', 8)
('LAS', 10)
('LAX', 28)
('LGA', 9)
('MCO', 1)
('MFR', 1)
('MIA', 2)
('MSP', 1)
('MSY', 3)
('OGG', 2)
('ORD', 30)
('PBI', 1)
('PDX', 1)
('PIT', 1)
('RDU', 1)
('ROC', 1)
('SAN', 1)
('SEA', 8)
('SFO', 42)
('SLC', 1)
('SNA', 1)
('TPA', 2)


**Join**
